In [8]:
# importing the required libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense

In [9]:
class TrafficSignNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (width, height, depth)
        chanDims = -1
        model.add(Conv2D(8, (5, 5) ,padding = "same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDims))
        model.add(MaxPooling2D(pool_size=(2,2)))
#           conv-> relu-> conv-> relu-> pool
        model.add(Conv2D(16, (3,3) , padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDims))
        model.add(Conv2D(16, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDims))
        model.add(MaxPooling2D(pool_size = (2,2)))
#           conv-> relu-> conv-> relu-> pool           
        model.add(Conv2D(32, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDims))
        model.add(Conv2D(32, (3,3) ,padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDims))
        model.add(MaxPooling2D(pool_size=(2,2)))
#           1st set of FC layer
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
#           2nd set of FC layer
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
#           Softmax Classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        
        return model

In [10]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from skimage import transform
from skimage import exposure
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import os

In [11]:
def load_split(basePath, csvPath):
	# initialize the list of data and labels
	data = []
	labels = []

	# load the contents of the CSV file, remove the first line (since
	# it contains the CSV header), and shuffle the rows (otherwise
	# all examples of a particular class will be in sequential order)
	rows = open(csvPath).read().strip().split("\n")[1:]
	random.shuffle(rows)

	# loop over the rows of the CSV file
	for (i, row) in enumerate(rows):
		# check to see if we should show a status update
		if i > 0 and i % 1000 == 0:
			print("[INFO] processed {} total images".format(i))

		# split the row into components and then grab the class ID
		# and image path
		(label, imagePath) = row.strip().split(",")[-2:]

		# derive the full path to the image file and load it
		imagePath = os.path.sep.join([basePath, imagePath])
		image = io.imread(imagePath)

		# resize the image to be 32x32 pixels, ignoring aspect ratio,
		# and then perform Contrast Limited Adaptive Histogram
		# Equalization (CLAHE)
		image = transform.resize(image, (32, 32))
		image = exposure.equalize_adapthist(image, clip_limit=0.1)

		# update the list of data and labels, respectively
		data.append(image)
		labels.append(int(label))

	# convert the data and labels to NumPy arrays
	data = np.array(data)
	labels = np.array(labels)

	# return a tuple of the data and labels
	return (data, labels)


In [12]:
# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 30
INIT_LR = 1e-3
BS = 64

In [13]:
# load the label names
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

In [14]:
base = "gtsrb-german-traffic-sign/"
trainPath = base + "Train.csv"
testPath = base + "Test.csv"

print("Start: Loading Dataset")
(trainX, trainY) = load_split(basePath= base, csvPath= trainPath)
(testX, testY) = load_split(basePath= base, csvPath= testPath)
print("End: Loaded Dataset")
print("TrainY : ", trainY)

Start: Loading Dataset
[INFO] processed 1000 total images
[INFO] processed 2000 total images
[INFO] processed 3000 total images
[INFO] processed 4000 total images
[INFO] processed 5000 total images
[INFO] processed 6000 total images
[INFO] processed 7000 total images
[INFO] processed 8000 total images
[INFO] processed 9000 total images
[INFO] processed 10000 total images
[INFO] processed 11000 total images
[INFO] processed 12000 total images
[INFO] processed 13000 total images
[INFO] processed 14000 total images
[INFO] processed 15000 total images
[INFO] processed 16000 total images
[INFO] processed 17000 total images
[INFO] processed 18000 total images
[INFO] processed 19000 total images
[INFO] processed 20000 total images
[INFO] processed 21000 total images
[INFO] processed 22000 total images
[INFO] processed 23000 total images
[INFO] processed 24000 total images
[INFO] processed 25000 total images
[INFO] processed 26000 total images
[INFO] processed 27000 total images
[INFO] process

In [15]:
# Scaling data
trainXF = trainX.astype("float32") / 255.0
testXF = testX.astype("float32") / 255.0

In [16]:
print(len(np.unique(trainY)))
print(np.unique(trainY))
print(trainY)

43
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
[ 3 32 12 ... 10 38  3]


In [17]:
# one-hot encode the training and testing labels
numLabels = len(np.unique(trainY))
trainYF = to_categorical(trainY, numLabels)
testYF = to_categorical(testY, numLabels)

In [18]:
print(trainY.sum(axis=0))

619047


In [19]:
# account for skew in the labeled data
classTotals = trainYF.sum(axis=0)
classWeight = classTotals.max() / classTotals


In [20]:
from sklearn.utils import class_weight
weight = class_weight.compute_class_weight('balanced', np.unique(trainY), trainY)
weight = {i : weight[i] for i in range(43)}
print(weight)

{0: 4.342081949058693, 1: 0.41073748166771423, 2: 0.4052609819121447, 3: 0.6466930562427841, 4: 0.46052384308198263, 5: 0.49023505876469115, 6: 2.1710409745293466, 7: 0.6332202842377261, 8: 0.6466930562427841, 9: 0.620297421294099, 10: 0.45365035288672917, 11: 0.6907857646229739, 12: 0.4342081949058693, 13: 0.4221468561584841, 14: 1.1690220632081096, 15: 1.447360649686231, 16: 2.1710409745293466, 17: 0.8214749633354285, 18: 0.7598643410852713, 19: 4.342081949058693, 20: 2.5328811369509046, 21: 2.7631430584918957, 22: 2.3380441264162193, 23: 1.7879160966712266, 24: 3.3771748492678726, 25: 0.6078914728682171, 26: 1.5197286821705427, 27: 3.7993217054263564, 28: 1.6885874246339363, 29: 3.3771748492678726, 30: 2.0263049095607237, 31: 1.1690220632081096, 32: 3.7993217054263564, 33: 1.3234212036318223, 34: 2.1710409745293466, 35: 0.7598643410852713, 36: 2.3380441264162193, 37: 4.342081949058693, 38: 0.44050106729580946, 39: 3.0394573643410854, 40: 2.5328811369509046, 41: 3.7993217054263564, 4

In [21]:
# temppppppppppppppppppppppp
from sklearn.utils import class_weight
weight = class_weight.compute_class_weight('balanced', np.unique(trainY), trainY)
weight = {i : weight[i] for i in range(43)}
print(weight)

{0: 4.342081949058693, 1: 0.41073748166771423, 2: 0.4052609819121447, 3: 0.6466930562427841, 4: 0.46052384308198263, 5: 0.49023505876469115, 6: 2.1710409745293466, 7: 0.6332202842377261, 8: 0.6466930562427841, 9: 0.620297421294099, 10: 0.45365035288672917, 11: 0.6907857646229739, 12: 0.4342081949058693, 13: 0.4221468561584841, 14: 1.1690220632081096, 15: 1.447360649686231, 16: 2.1710409745293466, 17: 0.8214749633354285, 18: 0.7598643410852713, 19: 4.342081949058693, 20: 2.5328811369509046, 21: 2.7631430584918957, 22: 2.3380441264162193, 23: 1.7879160966712266, 24: 3.3771748492678726, 25: 0.6078914728682171, 26: 1.5197286821705427, 27: 3.7993217054263564, 28: 1.6885874246339363, 29: 3.3771748492678726, 30: 2.0263049095607237, 31: 1.1690220632081096, 32: 3.7993217054263564, 33: 1.3234212036318223, 34: 2.1710409745293466, 35: 0.7598643410852713, 36: 2.3380441264162193, 37: 4.342081949058693, 38: 0.44050106729580946, 39: 3.0394573643410854, 40: 2.5328811369509046, 41: 3.7993217054263564, 4

In [22]:
ActualWeight = {}
for (i,v) in enumerate(classWeight):
    ActualWeight[i] = v
print(ActualWeight)    

{0: 10.714286, 1: 1.0135136, 2: 1.0, 3: 1.5957447, 4: 1.1363636, 5: 1.2096775, 6: 5.357143, 7: 1.5625, 8: 1.5957447, 9: 1.5306122, 10: 1.119403, 11: 1.7045455, 12: 1.0714285, 13: 1.0416666, 14: 2.8846154, 15: 3.5714285, 16: 5.357143, 17: 2.0270271, 18: 1.875, 19: 10.714286, 20: 6.25, 21: 6.818182, 22: 5.769231, 23: 4.4117646, 24: 8.333333, 25: 1.5, 26: 3.75, 27: 9.375, 28: 4.1666665, 29: 8.333333, 30: 5.0, 31: 2.8846154, 32: 9.375, 33: 3.2656024, 34: 5.357143, 35: 1.875, 36: 5.769231, 37: 10.714286, 38: 1.0869565, 39: 7.5, 40: 6.25, 41: 9.375, 42: 9.375}


In [23]:
print(classWeight)

[10.714286   1.0135136  1.         1.5957447  1.1363636  1.2096775
  5.357143   1.5625     1.5957447  1.5306122  1.119403   1.7045455
  1.0714285  1.0416666  2.8846154  3.5714285  5.357143   2.0270271
  1.875     10.714286   6.25       6.818182   5.769231   4.4117646
  8.333333   1.5        3.75       9.375      4.1666665  8.333333
  5.         2.8846154  9.375      3.2656024  5.357143   1.875
  5.769231  10.714286   1.0869565  7.5        6.25       9.375
  9.375    ]


In [24]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=10,
	zoom_range=0.15,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	vertical_flip=False,
	fill_mode="nearest")


In [30]:
# compiling model
NUM_EPOCHS = 5
print("Start: Compile Model")
opt = Adam(lr=INIT_LR, decay=INIT_LR / (NUM_EPOCHS * 0.5))
model = TrafficSignNet.build(width = 32, height=32, depth=3, classes=numLabels)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics = ["accuracy"])
print("End: Compile Model")

Start: Compile Model
End: Compile Model


In [31]:
# training the networ Training of network")
print("Start: Training of network")
H = model.fit_generator(
	aug.flow(trainXF, trainYF, batch_size=BS),
	validation_data=(testXF, testYF),
	steps_per_epoch=trainXF.shape[0] // BS,
	epochs=NUM_EPOCHS,
# 	class_weight=ActualWeight,
	verbose=1)
print("End: Training of network")

Start: Training of network
Epoch 1/5
612/612 [==============================] - 46s 75ms/step - loss: 2.6515 - accuracy: 0.2922 - val_loss: 2.7415 - val_accuracy: 0.2298
Epoch 2/5
612/612 [==============================] - 49s 79ms/step - loss: 1.4798 - accuracy: 0.5269 - val_loss: 13.0393 - val_accuracy: 0.0625
Epoch 3/5
612/612 [==============================] - 51s 84ms/step - loss: 1.0661 - accuracy: 0.6568 - val_loss: 7.7383 - val_accuracy: 0.1279
Epoch 4/5
612/612 [==============================] - 49s 80ms/step - loss: 0.8210 - accuracy: 0.7368 - val_loss: 14.5826 - val_accuracy: 0.0283
Epoch 5/5
612/612 [==============================] - 74s 121ms/step - loss: 0.6868 - accuracy: 0.7763 - val_loss: 2.1141 - val_accuracy: 0.4706
End: Training of network


In [32]:
# Evaluatiing the network on the test set
print("Start: Evaluate network")
predictions = model.predict(testXF, batch_size=BS)
print(classification_report(testYF.argmax(axis=1), predictions.argmax(axis=1),target_names=labelNames))
print("End: Evaluate network")

Start: Evaluate network
                                                    precision    recall  f1-score   support

                              Speed limit (20km/h)       0.00      0.00      0.00        60
                              Speed limit (30km/h)       0.86      0.19      0.32       720
                              Speed limit (50km/h)       0.89      0.08      0.15       750
                              Speed limit (60km/h)       0.75      0.17      0.28       450
                              Speed limit (70km/h)       0.68      0.19      0.29       660
                              Speed limit (80km/h)       0.36      0.27      0.31       630
                       End of speed limit (80km/h)       0.30      0.21      0.25       150
                             Speed limit (100km/h)       0.21      0.36      0.27       450
                             Speed limit (120km/h)       0.16      0.68      0.26       450
                                        No passing     

/home/parthjardosh/anaconda3/envs/TSC/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# saving the model
print("Start: Save the model")
model.save("output/trafficsignnet" + str(NUM_EPOCHS) +".model")
print("End: Save the model")

Start: Save the model


In [ ]:
# plot the training loss and accuracy
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("output/plot" + str(NUM_EPOCHS) +".png")